DR-CG-Net (v 0.7.0)

Created by Carter Lyons

Last Updated 9/13/2023

* Seperated configuring network into seperate (config.py) file

# Create a DR-CG-Net for testing and visualization

In [1]:
import tensorflow as tf
import tensorflow.keras.initializers as Init
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from config import conf
from utils import LoadMatrices

In [2]:
img_size = 32
msmnt_type = 'angles'
msmnt = 15
noise = 60
num_training_samples = 20
load_weights = True # de-/activates loading trained network weights
Load = LoadMatrices.Load()

In [3]:
Psi = Load.load_Psi(img_size, msmnt, msmnt_type)
Phi, Phi_Wavelet = Load.load_Phi(img_size, conf.train.use_sparsity, conf.train.sparsity_type, conf.data.true_data_type)
A = tf.constant(Psi, dtype = 'float32') if Phi is None else tf.constant(Psi@Phi, dtype = 'float32')
if 'GDu' in conf.model.method:
    A_norm = tf.linalg.norm(A, ord=2)
    A = tf.sparse.from_dense(A)

In [4]:
# Create Network
import tensorflow.keras.initializers as Init
if 'GDu' in conf.model.method:
    from networks import NewDRCGNetSparseGDu as Net
else:
    from networks import NewDRCGNet as Net

cov_params = {'const': [Init.Constant(conf.model.lamb_init[noise])],
              'diag': [Init.Constant(conf.model.lamb_init[noise])],
              'tridiag': [Init.Constant(conf.model.lamb_init[noise]), Init.Constant(0.0), 'SRDD']
             }
if conf.model.cov_style == 'full':
    n = img_size**2
    Q = np.zeros((n*(n+1)//2,))
    indx = np.array([i*(n+1)+1 for i in range(n-n//2)]+[i*(n+1) for i in range(n-n//2-n%2, 0, -1)])-1
    Q[indx] = conf.model.lamb_init[noise]*np.ones(n)
    cov_params[conf.model.cov_style] = [Init.Constant(Q), 'DEFAULT'] 
    del Q 

if 'GDu' in conf.model.method:
    model = Net.DRCGNet(
          conf.model.K, 
          conf.model.J, 
          conf.model.GDsteps,
          A, 
          A_norm,
          img_size, 
          conf.model.CNN_option, 
          cov_params[conf.model.cov_style], 
          conf.model.cov_style,
          scale_cov = conf.model.scale_cov,
          u0_style = conf.model.u0_style, 
          normalize_init = conf.model.normalize_A_in_z0,
          b = conf.model.b,
          D = conf.model.D,
          filters = conf.model.num_filters,
          kernel_size = conf.model.kernel_size,
          eta = conf.model.step_size,
          normalize_grad = conf.model.normalize_grad,
          B = conf.model.B,
          eps = conf.model.epsilon,                      
          bias = conf.model.use_bias,
          project = conf.model.project_z_est,
          denoise = conf.model.use_refinement,
          shared_u_weights = conf.model.shared_u_weights)
else:
    model = Net.DRCGNet(
              conf.model.K, 
              conf.model.J, 
              A, 
              img_size, 
              conf.model.CNN_option, 
              cov_params[conf.model.cov_style], 
              conf.model.cov_style,
              scale_cov = conf.model.scale_cov,
              u0_style = conf.model.u0_style, 
              normalize_init = conf.model.normalize_A_in_z0,
              b = conf.model.b,
              D = conf.model.D,
              filters = conf.model.num_filters,
              kernel_size = conf.model.kernel_size,
              eta = conf.model.step_size,
              normalize_grad = conf.model.normalize_grad,
              B = conf.model.B,
              eps = conf.model.epsilon,                      
              bias = conf.model.use_bias,
              project = conf.model.project_z_est,
              denoise = conf.model.use_refinement)

model = model.call()

In [ ]:
# model.summary()

In [10]:
if load_weights:
    if img_size == 32:
        model.load_weights(os.path.join('Weights', '{}_{}{}_{}dB_{}_samples.h5'.format(conf.general.SaveName, msmnt, msmnt_type, noise, num_training_samples)))
    else:
        model.load_weights(os.path.join('Weights', '{}_{}{}_{}dB_{}_samples_{}_imgsize.h5'.format(conf.general.SaveName, msmnt, msmnt_type, noise, num_training_samples, img_size)))

In [5]:
# Load test dataset
path_to_test_data = os.path.join('Test Datasets', '{}x{}_{}{}_{}dB_test_data.csv'.format(img_size, img_size, msmnt, msmnt_type, noise))
test_data = pd.read_csv(path_to_test_data)
test_data.drop(test_data.columns[0], axis = 1, inplace = True)
test_data = np.array(test_data)

In [6]:
# Reconstruct Examples
i = 0
batch_size = 2
m, n = tf.shape(A)
sample = test_data[i:i+batch_size]
c_act, y = sample[:,m:], sample[:,:m]
I_act = tf.constant(np.reshape(c_act, (batch_size, img_size, img_size, 1)), dtype = 'float32') if Phi_Wavelet is None else tf.constant(np.reshape(c_act@Phi_Wavelet.T, (batch_size, img_size, img_size, 1)), dtype = 'float32')
x_est = model(y)
x_est = tf.reshape(x_est, (batch_size, img_size, img_size, 1)) if Phi is None else tf.reshape(x_est@tf.cast(tf.transpose(Phi), dtype = 'float32'), (batch_size, img_size, img_size, 1)) 
ssims = tf.image.ssim(I_act, x_est, max_val = conf.data.scale)
psnrs = tf.image.psnr(I_act, x_est, max_val = conf.data.scale)

In [8]:
# Display examples
i = 1

fig,axes = plt.subplots(1,2)

axes[0].imshow(np.reshape(x_est[i], (img_size,img_size)), cmap = 'gray')
axes[0].set_xlabel('Estimate')

axes[1].imshow(np.reshape(I_act[i], (img_size, img_size)), cmap = 'gray')
axes[1].set_xlabel('Actual')

plt.show()
print('SSIM {:.5f} & PSNR {:.3f}'.format(ssims[i], psnrs[i]))

# MultiRun

Creates, trains, and tests multiple DR-CG-Nets

In [1]:
import tensorflow as tf

from config import conf
from utils import MultiRun
if 'GDu' in conf.model.method:
    from networks import NewDRCGNetSparseGDu as Net
else:
    from networks import NewDRCGNet as Net

In [3]:
img_size = 32  # 32 or 64

# For Radon inversion experiments
msmnt_type = 'angles'
msmnt_list = [15]  # List of the number of angles used in the radon transform.
noise_list = [[60], [60], [60]]  # Measurement SNR in dB. Should be a list of lists. noise_list[i][j] = measurement SNR to train a DR-CG-Net with for msmnt_list[i] number of angles.
training_list = [[[20]], [[20, 100]], [[20, 100]]]  # Number of samples to train with. Should be triple nested lists. training_list[i][j][k] = number of samples to train a DR-CG-Net with for noise_list[i][j] measurement SNR and msmsnt_list[i] number of angles.
epoch_list = [[[2000]], [[2000, 2000]], [[2000, 2000]]]  # Maximum number of training epochs. Should be triple nest lists. epoch_list[i][j][k] = maximum number of epochs for training a DR-CG-Net using training_list[i][j][k] samples with noise_list[i][j] measurement SNR and msmsnt_list[i] number of angles.

# # For CS experiments
# msmnt_type = 'ratio'
# msmnt_list = [10,30,50]  # List of the sampling ratios (x 100) used in the Gaussian matrix
# noise_list = [[60],[60],[60]]
# training_list = [[[20]],[[20]],[[20]]]
# epoch_list = [[[2000]],[[2000]],[[2000]]]

mr = MultiRun.MultiRun(Net.DRCGNet, Net.Grad())
mr.call(img_size, msmnt_list, noise_list, training_list, epoch_list, msmnt_type)